In [5]:
%matplotlib inline
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tq
from PIL import Image
import keras
import pickle
from keras.models import load_model
from keras.models import Sequential
from keras.models import Model
from keras.utils.layer_utils import layer_from_config
from keras.layers import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import load_img, img_to_array, random_rotation, flip_axis, array_to_img
import itertools
import gc
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input as preprocess_input_inception
from keras.models import model_from_json
from keras.applications.inception_v3 import InceptionV3
from heatmap import to_heatmap
from heatmap import synset_to_dfs_ids
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing import image
import scipy
from heatmaps_generator import *
import os
import glob
# To see the training error in real time.
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://localhost:9000')

Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


In [6]:
def change_resolution_25p_up(x):
    x = array_to_img(x)
    size = [int(a*1.25) for a in x.size] 
    x = scipy.misc.imresize(x, size[::-1], interp='bilinear', mode=None)
    return image.img_to_array(x)

def change_resolution_25p_down(x):
    x = array_to_img(x)
    size = [int(a*0.75) for a in x.size] 
    x = scipy.misc.imresize(x, size[::-1], interp='bilinear', mode=None)
    return image.img_to_array(x)

def flip_horizontal_img(x):
    return flip_axis(x, 2)
def flip_horizontal_hm(x):
    return flip_axis(x, 1)

def flip_vertical_img(x):
    return flip_axis(x, 1)
def flip_vertical_hm(x):
    return flip_axis(x, 0)

def rotate_img_15_up(x):
    x = array_to_img(x)
    x = x.rotate(15)
    return img_to_array(x)
def rotate_hm_15_down(x):
    x = array_to_img(np.expand_dims(x, axis=0))
    x = x.rotate(-15)
    return img_to_array(x)[0]/255

In [7]:
get_all_images()[0][14:].split("/")[0]

'ALB'

In [8]:
def generate_heatmaps(models, models_names,transformations, inverses, transformations_names):
    mk("heatmaps")
    mk("heatmaps/pickle")
    mk("heatmaps/png")
    
    paths_list = get_all_images()
    
    for model_function, model_name in zip(models, models_names):
        model = model_function()
        model = to_heatmap(model)
        
        mk("heatmaps/pickle/" + model_name)
        mk("heatmaps/png/" + model_name)
        
        
        for path in tq(paths_list):
        
            for transformation, inverse, tr_name in zip(transformations, inverses, transformations_names):
            
                mk("heatmaps/pickle/" + model_name + "/" + tr_name)
                mk("heatmaps/png/" + model_name + "/" + tr_name)
                mk("heatmaps/png/" + model_name + "/" + tr_name +"/"+ path[13:].split("/")[1])
                mk("heatmaps/pickle/" + model_name + "/" + tr_name +"/"+ path[13:].split("/")[1])
                
                pickle_path = "heatmaps/pickle/" + model_name + "/" + tr_name +  path[13:]
                png_path = "heatmaps/png/" + model_name + "/" + tr_name +  path[13:]
                
                if not os.path.isfile(pickle_path) or not os.path.isfile(png_path):
                    img = image.load_img(path)
                    img = image.img_to_array(img)
                    h = get_heatmap_from_transformation(img, model, transformation, inverse)
                    with open(pickle_path, "wb") as f:
                        pickle.dump(h, f, protocol=pickle.HIGHEST_PROTOCOL)
                    
                    h=h*255
                    plt.imsave(png_path,h)
                    
            
    

In [9]:
#models=[VGG16,VGG19, ResNet50][::-1]
#models_names = ["VGG16","VGG19", "ResNet50"][::-1]
models=[VGG19][::-1]
models_names = ["VGG19"][::-1]
transformations = [lambda x:x, 
                   flip_horizontal_img,
                   flip_vertical_img,
                   rotate_img_15_up,
                   change_resolution_25p_up,
                   change_resolution_25p_down]
inverses = [lambda x:x, 
            flip_horizontal_hm,
            flip_vertical_hm,
            rotate_hm_15_down,
            lambda x: x,
            lambda x: x]
transformations_names = ["identity", 
                         "horizontal flip",
                         "vertical flip",
                         "rotate 15",
                         "resolution 25 up",
                         "resolution 25 down"]

In [10]:
gc.collect()
generate_heatmaps(models, models_names,transformations, inverses, transformations_names)

Model type detected: local pooling - flatten
Model cut at layer: 21
Pool size infered: 7
New convolution size: 13


MemoryError: ('Error allocating 1417674752 bytes of device memory (CNMEM_STATUS_OUT_OF_MEMORY).', "you might consider using 'theano.shared(..., borrow=True)'")

Let's try to make a neural network guess the weights repartitions:

In [ ]:
_dir = "heatmaps/pickle/ResNet50/horizontal flip/ALB"
list1 = []
for i in list(os.listdir(_dir)):
    with open(_dir + "/" + i, "rb") as f:
        h = pickle.load(f)
    list1.append(h)
_dir = "heatmaps/pickle/VGG16/horizontal flip/ALB"
list2 = []
for i in list(os.listdir(_dir)):
    with open(_dir + "/" + i, "rb") as f:
        h = pickle.load(f)
    list2.append(h)
_dir = "heatmaps/pickle/VGG16/identity/ALB"
list3 = []
for i in list(os.listdir(_dir)):
    with open(_dir + "/" + i, "rb") as f:
        h = pickle.load(f)
    list3.append(h)

In [ ]:
print(len(list1))
print(len(list2))
len(list3)

In [ ]:
list1[0].reshape((1,100*100))[0]

In [ ]:
X = []

for h1,h2,h3 in zip(list1,list2,list3):
    l1 = h1.reshape((1,100*100))[0]
    l2 = h2.reshape((1,100*100))[0]
    l3 = h3.reshape((1,100*100))[0]
    
    for i in range(100*100):
        X.append([l1[i],l2[i],l3[i]])#h1,h2,

In [ ]:
X = []

for h1,h2,h3 in zip(list1,list2,list3):
    X.append([h1,h2,h3])

In [ ]:
X=np.array(X)
X.shape

In [ ]:
l = len(X)
list_masks = []

for i in tq(list(os.listdir(_dir))[:l]):
    mask = get_mask_from_img("ALB",i, resolution("train/ALB/" + i))
    l = mask.reshape((1,100*100))[0]
    list_masks.append([mask])
    #list_masks+=list(l)

In [ ]:
Y = np.array(list_masks)

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
Y = Y/2

In [ ]:
nb_maps=3
img_input = Input(shape=((nb_maps,)))
x = Dense(1,bias=False, activation="softmax") (img_input)
model = Model(img_input, x)

In [ ]:
from keras.regularizers import l2, l1
nb_maps=3
img_input = Input(shape=(nb_maps,100,100))
x = Convolution2D(1,1,1, bias=False, W_regularizer=l2(0.0001)) (img_input)
x = Convolution2D(5,3,3, border_mode="same", W_regularizer=l2(0.0001)) (x)
x = Convolution2D(1,3,3, border_mode="same",W_regularizer=l2(0.0001)) (x)
x = Convolution2D(1,1,1, border_mode="same", activation="sigmoid") (x)
# , W_regularizer=l2(0.01)
model = Model(img_input, x)

In [ ]:
model.layers[1].get_weights()

In [ ]:
c=0
w = np.array([[c],[c],[c]])#
model.layers[1].set_weights([w])

In [ ]:
len(Y)/np.sum(Y)

In [ ]:
c=0.5
w = np.array([[[[c]],[[c]],[[c]]]])#
model.layers[1].set_weights([w])

In [ ]:
sgd = keras.optimizers.SGD(lr=0.03, momentum=0.0, decay=0.0, nesterov=False)
model.compile(optimizer=sgd,loss='mse')

In [ ]:
class_weight = {0 : 1, 1: 23.1}

In [ ]:
model.fit(X, Y, verbose=0, nb_epoch=200, batch_size=46,validation_split=0.2, callbacks=[remote])

In [ ]:
Z = model.predict(X)

In [ ]:
plt.imshow(X[-2][0], interpolation=None)
plt.show()

In [ ]:
plt.imshow(X[-2][1], interpolation=None)
plt.show()

In [ ]:
plt.imshow(X[-2][2], interpolation=None)
plt.show()

In [ ]:
plt.imshow(Y[-2][0], interpolation=None)
plt.show()

In [ ]:
plt.imshow(Z[-2][0], interpolation=None)
plt.show()

In [ ]:
np.min(X[0][0])

In [ ]:
np.max(X[0][0])

In [ ]:
np.min(Y[0][0])

In [ ]:
np.max(Y[0][0])

In [ ]:
np.min(Z[-1][0])

In [ ]:
np.max(Z[-1][0])

In [ ]:
model.summary()

In [ ]:
Y.shape

In [ ]:
np.max(mask)

In [ ]:
model = VGG16()
model = to_heatmap(model)

In [ ]:
img_path = "./train/train/BET/img_02016.jpg"
img = image.load_img(img_path)
img = image.img_to_array(img)
h1 = get_heatmap_from_transformation(img, model, debug=True)

In [ ]:
h2 = get_heatmap_from_transformation(img, model, flip_horizontal_img, flip_horizontal_hm, debug=True)

In [ ]:
plt.imshow((h1+h2)/2, interpolation="none")
plt.show()

In [ ]:
model=ResNet50()
model = to_heatmap(model)

In [ ]:
h3 = get_heatmap_from_transformation(img, model, last=True)

In [ ]:
plt.imshow((h1+h3)/2, interpolation="none")
plt.show()

In [ ]:
model=VGG19()
model = to_heatmap(model)

In [ ]:
h4 = get_heatmap_from_transformation(img, model, last=True)

In [ ]:
plt.imshow((h1+h3+h4)/3, interpolation="none")
plt.show()

In [ ]:
h5 = get_heatmap_from_transformation(img, model,increase_resolution, debug=True)

In [ ]:
plt.imshow(scipy.misc.imresize((h1+h2+h3+h4+h5*2)/6, (720,1280), interp='bilinear', mode=None), interpolation="none")
plt.show()

In [ ]:
from skimage import exposure
p2, p98 = np.percentile(scipy.misc.imresize((h1+h2+h3+h4+h5*2)/6, (720,1280), interp='bilinear', mode=None), (0.1, 99.9))
new_heatmap = exposure.rescale_intensity(scipy.misc.imresize((h1+h2+h3+h4+h5*2)/6, (720,1280), interp='bilinear', mode=None), in_range=(p2, p98))
plt.imshow(new_heatmap)
plt.show()

In [ ]:
def 